# Imports

In [ ]:
# !pip -q install ktrain requests

In [ ]:
!python -m spacy download pt_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-10 22:57:31.002118: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import pandas as pd
import numpy as np
import json
import pickle as pkl

import yaml

# import ktrain 
# from ktrain import text as txt

import uuid

import re
import spacy
from spacy.lang.pt.examples import sentences 
from spacy.training import offsets_to_biluo_tags



/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Utils

In [ ]:
nlp = spacy.load('pt_core_news_sm')

entidadesExtratoContrato = ['data_assinatura_contrato',
                            'partes_contratuais',
                            'natureza_despesa',
                            'nota_empenho',
                            'numero_contrato',
                            'objeto_contrato',
                            'processo_gdf',
                            'programa_trabalho',
                            'unidade_orcamentaria',
                            'valor_contrato',
                            'vigencia_contrato'
                            ]

def get_token_for_char(doc, char_idx):
    for i, token in enumerate(doc):
        if  token.idx <= char_idx <= token.idx+len(token.text):
            return token.idx,len(token.text)

def get_span(text, label, ent):
    span = {'label':label, 'start':0, 'end':0}
    doc = nlp(text)
    if ent in doc.text:
        t1 = get_token_for_char(doc,doc.text.index(ent))
        t2 = get_token_for_char(doc,doc.text.index(ent)+len(ent))
        span['start'] = t1[0]
        span['end'] = t2[0]+t2[1]
        #print(doc.text[span['start']:span['end']])
    return span

def entity_to_span(text, value, key):
    span_list = []
    e=0
    text = re.sub('\n', ' ', text)
    for entity in value:
        doc = nlp(text)
        ent = re.sub('\n', ' ', entity.replace(r'\n', '\n'))
        #print(text, '\n\t\t',key,'->', ent,'\n\n')
        #print(ent)
        span = {'label':key, 'start':0, 'end':0}
        #ent = entity[0]
        if ent in doc.text:
            #print(ent)
            #print(doc.text)
            t1 = get_token_for_char(doc,doc.text.index(ent))
            #print(doc.text.index(ent)+len(ent)-1)
            t2 = get_token_for_char(doc,doc.text.index(ent)+len(ent))
            span['start'] = t1[0]
            span['end'] = t2[0]+t2[1]
            #print(doc.text[span['start']:span['end']])
            if(span['start']!=0 and span['end']!=0):
                span_list.append((span['start']+e,span['end']+e,span['label']))
            e+=span['end']
            text = text[span['end']:]
    return span_list

# testando prediçoes coflitantes
# '(948, 958, 'data_assinatura_contrato')' and '(907, 1101, 'vigencia_contrato')'
def ajust_preds(text,preds,preds2):
    #print(preds)
    text = re.sub('\n', ' ', text)
    for p1 in preds:
        for p2 in preds:
            #print(p1,p2)
            if ((p2[0]<=p1[0] and p1[0]<=p2[1]) or (p2[0]<=p1[1] and p1[1]<=p2[1])) and p1 != p2:
                # print('p1:', p1)
                # print('p2:', p2)
                ent = text[p1[0]:p1[1]]
                label = p1[2]
                #print(ent)
                text_aux = text[p2[1]:]
                idx0 = p2[0]
                idx1 = p2[1]
                flag = 0
                # Procura no texto por outra ocorrencia
                while flag == 0:
                    flag = 1
                    span = get_span(text_aux,label,ent)
                    #print(span)
                    for p3 in preds:
                        # testa se esta occorencia ja está em alguma outra entidade, se sim continua procurando, se não encontrou a ocorrencia desejada
                        if ((p3[0]<=(span['start']+idx1)<=p3[1]) or (p3[0]<=(span['end']+idx1)<=p3[1])) and (span['start']!=0 and span['end']!=0):
                            #print('p3[0]:',p3[0])
                            #print('p3[1]:',p3[1])
                            flag = 0
                            text_aux = text[p3[1]:]
                            #print(text_aux)
                            idx0 = p3[0]
                            idx1 = p3[1]
                            # print(f"{span['start']} + {idx1} = {span['start']+idx1}")
                            # print(f"{span['end']} + {idx1} = {span['end']+idx1}")
                            #break
                # se achou um acorrencia que não faz parte de outra entidade, remove sua ocorrencia anterior e adiciona a nova
                if(span['start']!=0 and span['end']!=0):
                    #print('trocou p1: ', p1)
                    #print('por: ', (span['start']+idx1,span['end']+idx1,span['label']))
                    preds.remove(p1)
                    preds.append((span['start']+idx1,span['end']+idx1,span['label']))
                    return preds,preds2,0
                # se não achou nenhuma nova ocorrencia, devemos retirar a ocorrencia de maior tamanho, pois é provavel que essa esteja afetando a ocorrencia menor
                else:
                    ent = text[p2[0]:p2[1]]
                    label = p2[2]
                    #print(ent)
                    text_aux = text[p1[1]:]
                    idx0 = p1[0]
                    idx1 = p1[1]
                    flag = 0
                    while flag == 0:
                        flag = 1
                        span = get_span(text_aux,label,ent)
                        #print(span)
                        for p3 in preds:
                            # testa se esta occorencia ja está em alguma outra entidade, se sim continua procurando, se não encontrou a ocorrencia desejada
                            if ((p3[0]<=(span['start']+idx1)<=p3[1]) or (p3[0]<=(span['end']+idx1)<=p3[1])) and (span['start']!=0 and span['end']!=0):
                                #print('p3[0]:',p3[0])
                                #print('p3[1]:',p3[1])
                                flag = 0
                                text_aux = text[p3[1]:]
                                #print(text_aux)
                                idx0 = p3[0]
                                idx1 = p3[1]
                                # print(f"{span['start']} + {idx1} = {span['start']+idx1}")
                                # print(f"{span['end']} + {idx1} = {span['end']+idx1}")
                                #break
                    if(span['start']!=0 and span['end']!=0):
                        #print('trocou p1: ', p1)
                        #print('por: ', (span['start']+idx1,span['end']+idx1,span['label']))
                        preds.remove(p2)
                        preds.append((span['start']+idx1,span['end']+idx1,span['label']))
                        return preds,preds2,0
                    else:
                        #print('removeu p1: ', p1)
                        preds2.append(p1)
                        preds.remove(p1)
                        return preds,preds2,0
    return preds,preds2,1

def mix_final_preds(final_preds):
    for i in range(len(final_preds)):
        for j in range(i,len(final_preds)):
            for p1 in final_preds[i]:
                flag = 1
                for p2 in final_preds[j]:
                    if ((p2[0]<=p1[0]<=p2[1]) or (p2[0]<=p1[1]<=p2[1]) or (p1[0]<=p2[0]<=p1[1]) or (p1[0]<=p2[1]<=p1[1])):
                        flag = 0
                if flag==1:
                    final_preds[j].append(p1)
    return final_preds

def rec_ajust_preds(text,preds):
    final_preds = []
    preds2 = []
    teste = 0
    while teste != 1:
        preds,preds2, teste = ajust_preds(text,preds,preds2)
    if(preds2==[]):
        return [preds]
    else:
        final_preds.append(preds)
        for pred in rec_ajust_preds(text,preds2):
            final_preds.append(pred)
        # Junta as predições que não sobresaeem uma da outra
        return mix_final_preds(final_preds)

def get_list_spans(text,labels_dict):
    #labels_dict = yaml.safe_load(labels_dict)
    lista = []
    for key, value in labels_dict.items():
        if key not in entidadesExtratoContrato:
          continue
        #pred = ajust_preds(text,entity_to_span(text,value,key))
        pred = entity_to_span(text,value,key)
        for i in pred:
            lista.append(i)
    #print(lista)
    return rec_ajust_preds(text,lista)

In [ ]:
def tokenize(texto,labels,index):
    if isinstance(labels, str):
        labels = yaml.safe_load(re.sub('\n', ' ', labels.replace(r'\n', '\n')))
        
    nlp = spacy.load('pt_core_news_sm')
    preds = get_list_spans(texto,labels)

    if(len(preds)>1):
        print(index)

    conll=''
    for pred in preds:

        text = re.sub('\n', ' ', texto)
        doc = nlp(text)

        tags = offsets_to_biluo_tags(doc, pred)

        for i in range(len(doc)):
            if tags[i][0] == 'U':
                conll += doc[i].text + ' ' + 'B' + tags[i][1:] + '\n'
            elif tags[i][0] == 'L':
                conll += doc[i].text + ' ' + 'I' + tags[i][1:] + '\n'
            else:
                conll += doc[i].text + ' ' + tags[i] + '\n'
              
        conll+='\n'

    return conll

In [ ]:
# Função que gera string no formato Conll a partir das predições dos modelos
def get_conll_conc(pred):
  conll_conc = ""
  for i, tupla in enumerate(pred):
      txt,ent = tupla[0],tupla[1]
      if ent =='O':
        conll_conc += txt + ' O\n'
      else:
        if(i!=0 and pred[i-1][1]==ent):
          conll_conc += txt + ' '+ ent +'\n'
        else:
          conll_conc += txt + ' '+ ent +'\n'

  return conll_conc+'\n'

In [ ]:
# função que trasnforma as prediçõe dos modelos ktrain em dicionarios para facilitar a comparação sem depender de tokenizações
def get_entities_from_prediction(predictions):
    lista = []
    cont = 0
    for pred in predictions:
        ents_dit = {}
        i=0
        while (i < len(pred)):
            txt,ent = pred[i][0],pred[i][1]
            if ent !='O':
                aux_txt = txt
                while(len(pred) > i+1 and pred[i+1][1] != 'O'):
                    if(pred[i+1][1][2:] == ent[2:]):
                        aux_txt += ' ' + pred[i+1][0]
                        i+=1
                    else:
                        break
                if(ent[2:] in ents_dit):
                    ents_dit[ent[2:]].append(aux_txt)
                else:
                    ents_dit[ent[2:]] = [aux_txt]
            i+=1
        lista.append(ents_dit)
        cont+=1
    return lista

# Carregando dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Ajustando df_old

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TccShow/df_extrato_contrato.csv', encoding= 'utf-8')

In [ ]:
df

,Unnamed: 0,Unnamed: 0.1,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato,label,label_txt,pred
0,194,194,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,EXTRATO_CONTRATO,448,anotador_73210084,556520,1834,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,12_22.11.2018-R1,"{'numero_contrato': ['45/2018'], 'processo_gdf...",numero_contrato: 45/2018 # processo_gdf: 00070...,{}
1,210,210,12_22.11.2018,REL_EXTRATO_CONTRATO,R2,anotador_73210084,EXTRATO_CONTRATO,450,anotador_73210084,558353,1847,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,12_22.11.2018-R2,"{'orgao_contratante': ['SEAGRI/DF'], 'entidade...",orgao_contratante: SEAGRI/DF # entidade_contra...,"{'fonte_recurso': ['332012027'], 'natureza_des..."
2,240,240,12_22.11.2018,REL_EXTRATO_CONTRATO,R3,anotador_73210084,EXTRATO_CONTRATO,399,anotador_73210084,572972,1187,EXTRATO DO CONTRATO No 32/2018\nAQUISICAO DE B...,12_22.11.2018-R3,"{'numero_contrato': ['32/2018'], 'processo_gdf...",numero_contrato: 32/2018 # processo_gdf: 00053...,{'orgao_contratante': ['INSTITUTO DE ASSIST. À...
3,297,297,12_22.11.2018,REL_EXTRATO_CONTRATO,R4,anotador_73210084,EXTRATO_CONTRATO,453,anotador_73210084,599347,2060,EXTRATO DE CONTRATO\nCONTRATO No 9020. ASSINAT...,12_22.11.2018-R4,"{'numero_contrato': ['9020'], 'data_assinatura...",numero_contrato: 9020 # data_assinatura_contra...,{}
4,371,371,12_22.11.2018,REL_EXTRATO_CONTRATO,R7,anotador_73210084,EXTRATO_CONTRATO,460,anotador_73210084,611856,1236,"EXTRATO DO CONTRATO No 01/2018,\nNOS TERMOS DO...",12_22.11.2018-R7,"{'numero_contrato': ['01/2018'], 'processo_gdf...",numero_contrato: 01/2018 # processo_gdf: 00367...,"{'fonte_recurso': ['100'], 'natureza_despesa':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,40914,40929,638,REL_EXTRATO_CONTRATO,R81,anotador_73210084,EXTRATO_CONTRATO,1824,anotador_73210084,178278,1049,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,638-R81,"{'data_assinatura_contrato': ['15/03/2022'], '...",data_assinatura_contrato: 15/03/2022 # orgao_c...,"{'orgao_contratante': ['SEEDF', 'SEEDF'], 'obj..."
1538,40924,40939,638,REL_EXTRATO_CONTRATO,R82,anotador_73210084,EXTRATO_CONTRATO,1815,anotador_73210084,179328,1822,EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...,638-R82,"{'orgao_contratante': ['SEEDF'], 'unidade_orca...",orgao_contratante: SEEDF # unidade_orcamentari...,{}
1539,40994,41009,638,REL_EXTRATO_CONTRATO,R83,anotador_73210084,EXTRATO_CONTRATO,1220,anotador_73210084,205946,1122,EXTRATO DO CONTRATO Nº 45.926/2022\nProcesso: ...,638-R83,"{'fonte_recurso': ['100'], 'data_assinatura_co...",fonte_recurso: 100 # data_assinatura_contrato:...,{}
1540,41008,41023,638,REL_EXTRATO_CONTRATO,R84,anotador_73210084,EXTRATO_CONTRATO,1206,anotador_73210084,207115,1267,EXTRATO DO CONTRATO Nº 45832/2022\nProcesso: 0...,638-R84,"{'fonte_recurso': ['100'], 'data_assinatura_co...",fonte_recurso: 100 # data_assinatura_contrato:...,"{'fonte_recurso': ['100'], 'natureza_despesa':..."


### Separando

In [ ]:
df_old = df.loc[df['pred'] != '{}']
df_new = df.loc[df['pred'] == '{}']

In [ ]:
df_old

,Unnamed: 0,Unnamed: 0.1,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato,label,label_txt,pred
1,210,210,12_22.11.2018,REL_EXTRATO_CONTRATO,R2,anotador_73210084,EXTRATO_CONTRATO,450,anotador_73210084,558353,1847,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,12_22.11.2018-R2,"{'orgao_contratante': ['SEAGRI/DF'], 'entidade...",orgao_contratante: SEAGRI/DF # entidade_contra...,"{'fonte_recurso': ['332012027'], 'natureza_des..."
2,240,240,12_22.11.2018,REL_EXTRATO_CONTRATO,R3,anotador_73210084,EXTRATO_CONTRATO,399,anotador_73210084,572972,1187,EXTRATO DO CONTRATO No 32/2018\nAQUISICAO DE B...,12_22.11.2018-R3,"{'numero_contrato': ['32/2018'], 'processo_gdf...",numero_contrato: 32/2018 # processo_gdf: 00053...,{'orgao_contratante': ['INSTITUTO DE ASSIST. À...
4,371,371,12_22.11.2018,REL_EXTRATO_CONTRATO,R7,anotador_73210084,EXTRATO_CONTRATO,460,anotador_73210084,611856,1236,"EXTRATO DO CONTRATO No 01/2018,\nNOS TERMOS DO...",12_22.11.2018-R7,"{'numero_contrato': ['01/2018'], 'processo_gdf...",numero_contrato: 01/2018 # processo_gdf: 00367...,"{'fonte_recurso': ['100'], 'natureza_despesa':..."
5,392,392,12_22.11.2018,REL_EXTRATO_CONTRATO,R5,anotador_73210084,EXTRATO_CONTRATO,456,anotador_73210084,615487,1311,EXTRATO DO CONTRATO PARA PRESTACAO DE SERVICOS...,12_22.11.2018-R5,"{'numero_contrato': ['07/2018'], 'processo_gdf...",numero_contrato: 07/2018 # processo_gdf: 0391-...,"{'fonte_recurso': ['157'], 'natureza_despesa':..."
6,408,408,12_22.11.2018,REL_EXTRATO_CONTRATO,R6,anotador_73210084,EXTRATO_CONTRATO,458,anotador_73210084,616797,1266,EXTRATO DO CONTRATO TEC2B SOLUTIONS No 15/2018...,12_22.11.2018-R6,"{'orgao_contratante': ['IBRAM/DF'], 'fonte_rec...",orgao_contratante: IBRAM/DF # fonte_recurso: 1...,"{'orgao_contratante': ['IBRAM/DF'], 'objeto_co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,40854,40869,638,REL_EXTRATO_CONTRATO,R75,anotador_73210084,EXTRATO_CONTRATO,1874,anotador_73210084,173375,696,EXTRATO DO CONTRATO BRB Nº 140/2022\nContratad...,638-R75,"{'orgao_contratante': ['BRB'], 'nome_responsav...",orgao_contratante: BRB # nome_responsavel: BRU...,"{'fonte_recurso': ['1'], 'natureza_despesa': [..."
1532,40861,40876,638,REL_EXTRATO_CONTRATO,R76,anotador_73210084,EXTRATO_CONTRATO,1850,anotador_73210084,174072,781,EXTRATO DO CONTRATO BRB Nº 141/2022\nContratad...,638-R76,"{'orgao_contratante': ['BRB'], 'nome_responsav...",orgao_contratante: BRB # nome_responsavel: BRU...,{'orgao_contratante': ['BRASILIA RESERVADO AO ...
1534,40877,40892,638,REL_EXTRATO_CONTRATO,R78,anotador_73210084,EXTRATO_CONTRATO,1842,anotador_73210084,175636,650,EXTRATO DO CONTRATO BRB Nº 159/2022\nContratad...,638-R78,"{'orgao_contratante': ['BRB'], 'nome_responsav...",orgao_contratante: BRB # nome_responsavel: BRU...,"{'fonte_recurso': ['1'], 'natureza_despesa': [..."
1537,40914,40929,638,REL_EXTRATO_CONTRATO,R81,anotador_73210084,EXTRATO_CONTRATO,1824,anotador_73210084,178278,1049,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,638-R81,"{'data_assinatura_contrato': ['15/03/2022'], '...",data_assinatura_contrato: 15/03/2022 # orgao_c...,"{'orgao_contratante': ['SEEDF', 'SEEDF'], 'obj..."


### ajustando df_new

In [ ]:
df_new

,Unnamed: 0,Unnamed: 0.1,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato,label,label_txt,pred
0,194,194,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,EXTRATO_CONTRATO,448,anotador_73210084,556520,1834,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,12_22.11.2018-R1,"{'numero_contrato': ['45/2018'], 'processo_gdf...",numero_contrato: 45/2018 # processo_gdf: 00070...,{}
3,297,297,12_22.11.2018,REL_EXTRATO_CONTRATO,R4,anotador_73210084,EXTRATO_CONTRATO,453,anotador_73210084,599347,2060,EXTRATO DE CONTRATO\nCONTRATO No 9020. ASSINAT...,12_22.11.2018-R4,"{'numero_contrato': ['9020'], 'data_assinatura...",numero_contrato: 9020 # data_assinatura_contra...,{}
7,1351,1351,4_12.7.2019,REL_EXTRATO_CONTRATO,R1,anotador_73210084,EXTRATO_CONTRATO,28,anotador_73210084,559394,830,EXTRATO DO CONTRATO BRB 105/2019\nContratante:...,4_12.7.2019-R1,"{'numero_contrato': ['105/2019'], 'orgao_contr...",numero_contrato: 105/2019 # orgao_contratante:...,{}
11,1581,1581,1_31.5.2019,REL_EXTRATO_CONTRATO,R2,anotador_73210084,EXTRATO_CONTRATO,102,anotador_73210084,1206303,1973,EXTRATOS CONTRATUAIS\nEspecie: Contrato no 018...,1_31.5.2019-R2,"{'numero_contrato': ['018/2019-SES/DF'], 'orga...",numero_contrato: 018/2019-SES/DF # orgao_contr...,{}
13,1640,1640,1_31.5.2019,REL_EXTRATO_CONTRATO,R4,anotador_73210084,EXTRATO_CONTRATO,160,anotador_73210084,1285879,2508,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,1_31.5.2019-R4,"{'processo_gdf': ['00070-00004138/2019-45'], '...",processo_gdf: 00070-00004138/2019-45 # orgao_c...,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,40886,40901,638,REL_EXTRATO_CONTRATO,R79,anotador_73210084,EXTRATO_CONTRATO,1838,anotador_73210084,176319,961,EXTRATO DO CONTRATO MÚLTIPLO DE PRESTAÇÃO DE S...,638-R79,"{'orgao_contratante': ['SEEDF'], 'data_assinat...",orgao_contratante: SEEDF # data_assinatura_con...,{}
1536,40900,40915,638,REL_EXTRATO_CONTRATO,R80,anotador_73210084,EXTRATO_CONTRATO,1833,anotador_73210084,177281,996,EXTRATO DO CONTRATO DE AQUISIÇÃO DE BENS Nº 17...,638-R80,"{'fonte_recurso': ['100'], 'data_assinatura_co...",fonte_recurso: 100 # data_assinatura_contrato:...,{}
1538,40924,40939,638,REL_EXTRATO_CONTRATO,R82,anotador_73210084,EXTRATO_CONTRATO,1815,anotador_73210084,179328,1822,EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...,638-R82,"{'orgao_contratante': ['SEEDF'], 'unidade_orca...",orgao_contratante: SEEDF # unidade_orcamentari...,{}
1539,40994,41009,638,REL_EXTRATO_CONTRATO,R83,anotador_73210084,EXTRATO_CONTRATO,1220,anotador_73210084,205946,1122,EXTRATO DO CONTRATO Nº 45.926/2022\nProcesso: ...,638-R83,"{'fonte_recurso': ['100'], 'data_assinatura_co...",fonte_recurso: 100 # data_assinatura_contrato:...,{}


###ultimos ajustes:

In [ ]:
#resetando index
df_old.reset_index(inplace=True,drop=True)
df_old["id"] = df_old.index

# escolhendo colunas uteis
df_old = df_old[['id','id_ato','texto','label','pred']]
df_old

<ipython-input-14-8c1f29b52b00>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_old["id"] = df_old.index


,id,id_ato,texto,label,pred
0,0,12_22.11.2018-R2,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'orgao_contratante': ['SEAGRI/DF'], 'entidade...","{'fonte_recurso': ['332012027'], 'natureza_des..."
1,1,12_22.11.2018-R3,EXTRATO DO CONTRATO No 32/2018\nAQUISICAO DE B...,"{'numero_contrato': ['32/2018'], 'processo_gdf...",{'orgao_contratante': ['INSTITUTO DE ASSIST. À...
2,2,12_22.11.2018-R7,"EXTRATO DO CONTRATO No 01/2018,\nNOS TERMOS DO...","{'numero_contrato': ['01/2018'], 'processo_gdf...","{'fonte_recurso': ['100'], 'natureza_despesa':..."
3,3,12_22.11.2018-R5,EXTRATO DO CONTRATO PARA PRESTACAO DE SERVICOS...,"{'numero_contrato': ['07/2018'], 'processo_gdf...","{'fonte_recurso': ['157'], 'natureza_despesa':..."
4,4,12_22.11.2018-R6,EXTRATO DO CONTRATO TEC2B SOLUTIONS No 15/2018...,"{'orgao_contratante': ['IBRAM/DF'], 'fonte_rec...","{'orgao_contratante': ['IBRAM/DF'], 'objeto_co..."
...,...,...,...,...,...
778,778,638-R75,EXTRATO DO CONTRATO BRB Nº 140/2022\nContratad...,"{'orgao_contratante': ['BRB'], 'nome_responsav...","{'fonte_recurso': ['1'], 'natureza_despesa': [..."
779,779,638-R76,EXTRATO DO CONTRATO BRB Nº 141/2022\nContratad...,"{'orgao_contratante': ['BRB'], 'nome_responsav...",{'orgao_contratante': ['BRASILIA RESERVADO AO ...
780,780,638-R78,EXTRATO DO CONTRATO BRB Nº 159/2022\nContratad...,"{'orgao_contratante': ['BRB'], 'nome_responsav...","{'fonte_recurso': ['1'], 'natureza_despesa': [..."
781,781,638-R81,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,"{'data_assinatura_contrato': ['15/03/2022'], '...","{'orgao_contratante': ['SEEDF', 'SEEDF'], 'obj..."


In [ ]:
#resetando index
df_new.reset_index(inplace=True,drop=True)
df_new["id"] = df_new.index

# escolhendo colunas uteis
df_new = df_new[['id','id_ato','texto','label']].copy()
#df_new = df_new.replace(r'\n',' ', regex=True) 
df_new

<ipython-input-15-8b259af4e05a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["id"] = df_new.index


,id,id_ato,texto,label
0,0,12_22.11.2018-R1,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'numero_contrato': ['45/2018'], 'processo_gdf..."
1,1,12_22.11.2018-R4,EXTRATO DE CONTRATO\nCONTRATO No 9020. ASSINAT...,"{'numero_contrato': ['9020'], 'data_assinatura..."
2,2,4_12.7.2019-R1,EXTRATO DO CONTRATO BRB 105/2019\nContratante:...,"{'numero_contrato': ['105/2019'], 'orgao_contr..."
3,3,1_31.5.2019-R2,EXTRATOS CONTRATUAIS\nEspecie: Contrato no 018...,"{'numero_contrato': ['018/2019-SES/DF'], 'orga..."
4,4,1_31.5.2019-R4,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'processo_gdf': ['00070-00004138/2019-45'], '..."
...,...,...,...,...
754,754,638-R79,EXTRATO DO CONTRATO MÚLTIPLO DE PRESTAÇÃO DE S...,"{'orgao_contratante': ['SEEDF'], 'data_assinat..."
755,755,638-R80,EXTRATO DO CONTRATO DE AQUISIÇÃO DE BENS Nº 17...,"{'fonte_recurso': ['100'], 'data_assinatura_co..."
756,756,638-R82,EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...,"{'orgao_contratante': ['SEEDF'], 'unidade_orca..."
757,757,638-R83,EXTRATO DO CONTRATO Nº 45.926/2022\nProcesso: ...,"{'fonte_recurso': ['100'], 'data_assinatura_co..."


### filtrand e tirando "\\n"

In [ ]:
df_cp_old = df_old.copy()
df_cp_f = df_new.copy()

In [ ]:
entidadesExtratoContrato = ['data_assinatura_contrato',
                            'partes_contratuais',
                            'natureza_despesa',
                            'nota_empenho',
                            'numero_contrato',
                            'objeto_contrato',
                            'processo_gdf',
                            'programa_trabalho',
                            'unidade_orcamentaria',
                            'valor_contrato',
                            'vigencia_contrato'
                            ]

def label_clean(dit):
    new_dit = {}
    if isinstance(dit, str):
        dit = yaml.safe_load(re.sub('\n', ' ', dit.replace(r'\n', '\n')))
    for key,value in dit.items():
        if key == 'orgao_contratante' or key == 'entidade_contratada':
            if 'partes_contratuais' in new_dit:
                for v in value:
                    new_dit['partes_contratuais'].append(re.sub('\n', ' ', v))
            else:
                new_dit['partes_contratuais'] = []
                for v in value:
                    new_dit['partes_contratuais'].append(re.sub('\n', ' ', v))
        elif key in entidadesExtratoContrato:
            new_dit[key] = []
            for v in value:
                new_dit[key].append(re.sub('\n', ' ', v))
    return new_dit
    

for index, row in df_cp_old.iterrows():
    df_cp_old.loc[index, 'texto'] = re.sub('\n', ' ', row['texto'])
    df_cp_old.loc[index, 'label'] = str(label_clean(row['label']))
    df_cp_old.loc[index, 'pred'] = str(label_clean(row['pred']))

for index, row in df_cp_f.iterrows():
    df_cp_f.loc[index, 'texto'] = re.sub('\n', ' ', row['texto'])
    df_cp_f.loc[index, 'label'] = str(label_clean(row['label']))

In [ ]:
df_cp_old

,id,id_ato,texto,label,pred
0,0,12_22.11.2018-R2,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'partes_contratuais': ['SEAGRI/DF', 'CIMAG C...","{'natureza_despesa': ['449052'], 'partes_contr..."
1,1,12_22.11.2018-R3,EXTRATO DO CONTRATO No 32/2018 AQUISICAO DE BE...,"{'numero_contrato': ['32/2018'], 'processo_gdf...",{'partes_contratuais': ['INSTITUTO DE ASSIST. ...
2,2,12_22.11.2018-R7,"EXTRATO DO CONTRATO No 01/2018, NOS TERMOS DO ...","{'numero_contrato': ['01/2018'], 'processo_gdf...","{'natureza_despesa': ['449051'], 'partes_contr..."
3,3,12_22.11.2018-R5,EXTRATO DO CONTRATO PARA PRESTACAO DE SERVICOS...,"{'numero_contrato': ['07/2018'], 'processo_gdf...","{'natureza_despesa': ['3.3.90.39.11'], 'partes..."
4,4,12_22.11.2018-R6,EXTRATO DO CONTRATO TEC2B SOLUTIONS No 15/2018...,"{'partes_contratuais': ['IBRAM/DF', 'TEC2B SOL...","{'partes_contratuais': ['IBRAM/DF'], 'objeto_c..."
...,...,...,...,...,...
778,778,638-R75,EXTRATO DO CONTRATO BRB Nº 140/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Prosegur Brasi...","{'natureza_despesa': ['4'], 'partes_contratuai..."
779,779,638-R76,EXTRATO DO CONTRATO BRB Nº 141/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Renaforte Serv...",{'partes_contratuais': ['BRASILIA RESERVADO AO...
780,780,638-R78,EXTRATO DO CONTRATO BRB Nº 159/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Russell Bedfor...","{'natureza_despesa': ['4'], 'partes_contratuai..."
781,781,638-R81,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,"{'data_assinatura_contrato': ['15/03/2022'], '...","{'partes_contratuais': ['SEEDF', 'SEEDF'], 'ob..."


In [ ]:
df_cp_old['texto'][0]

'EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No 47/2018 PROCESSO: 00070-00016822/2018-99.  Partes: SEAGRI/DF  e CIMAG  COMERCIO DE IMPLEMENTOS E MAQUINAS AGRICOLAS LTDA. Objeto: O Contrato tem por objeto a aquisicao dos seguintes itens: Item 20: Quantidade 05 (cinco). GRADE ARADORA CONTROLE REMOTO, com 14 discos de 26 polegadas de diametro e 06 mm de espessura minima, mancais de rolamento com lubrificacao permanente em banho de oleo, ou a graxa, espacamento minimo entre os discos de 230mm. controle remoto para regulagem de profundidade do trabalho e transporte por meio do sistema hidraulico e pneus agricolas largura de corte minima de 1500 mm. compativel com tratores de potencia minima de 75 CV; Marca: CIMAG; Modelo / Versao: GRADE ARADORA CIMAG 14 x 26 x 6,0, consoante especifica o Edital de Pregao Eletronico no 04/2018 (Doc. Sei id 5507666) e a Proposta (Doc. Sei id 14816930), que passam a integrar o presente Termo. Valor: O valor total do Contrato e de R$ 66.000,00 (sessenta e seis m

In [ ]:
df_cp_old['pred'][0]

"{'natureza_despesa': ['449052'], 'partes_contratuais': ['CIMAG COMERCIO DE IMPLEMENTOS E MAQUINAS AGRICOLAS LTDA', 'SEAGRI/DF'], 'numero_contrato': ['47/2018'], 'processo_gdf': ['00070-00016822/2018-99'], 'objeto_contrato': ['O Contrato tem por objeto a aquisicao dos seguintes itens: Item 20: Quantidade 05 (cinco). GRADE ARADORA CONTROLE REMOTO, com 14 discos de 26 polegadas de diametro e 06 mm de espessura minima, mancais de rolamento com lubrificacao permanente em banho de oleo, ou a graxa, espacamento minimo entre os discos de 230mm. controle remoto para regulagem de profundidade do trabalho e transporte por meio do sistema hidraulico e pneus agricolas largura de corte minima de 1500 mm. compativel com tratores de potencia minima de 75 CV; Marca: CIMAG; Modelo / Versao: GRADE ARADORA CIMAG 14 x 26 x 6,0, consoante especifica o Edital de Pregao Eletronico no 04/2018 (Doc. Sei id 5507666) e a Proposta (Doc. Sei id']}"

In [ ]:
df_cp_old['label'][0]

"{'partes_contratuais': ['SEAGRI/DF', 'CIMAG  COMERCIO DE IMPLEMENTOS E MAQUINAS AGRICOLAS LTDA'], 'unidade_orcamentaria': ['14101'], 'programa_trabalho': ['20.606.6207.2889.0003'], 'natureza_despesa': ['449052'], 'data_assinatura_contrato': ['21/11/2018'], 'numero_contrato': ['47/2018'], 'processo_gdf': ['00070-00016822/2018-99'], 'valor_contrato': ['66.000,00'], 'nota_empenho': ['2018NE00514', '2018NE00515'], 'vigencia_contrato': ['O contrato tera vigencia desde a sua assinatura por ate 12 meses'], 'objeto_contrato': ['O Contrato tem por objeto a aquisicao dos seguintes itens: Item 20: Quantidade 05 (cinco). GRADE ARADORA CONTROLE REMOTO, com 14 discos de 26 polegadas de diametro e 06 mm de espessura minima, mancais de rolamento com lubrificacao permanente em banho de oleo, ou a graxa, espacamento minimo entre os discos de 230mm. controle remoto para regulagem de profundidade do trabalho e transporte por meio do sistema hidraulico e pneus agricolas largura de corte minima de 1500 mm.

In [ ]:
df_cp_f

,id,id_ato,texto,label
0,0,12_22.11.2018-R1,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'numero_contrato': ['45/2018'], 'processo_gdf..."
1,1,12_22.11.2018-R4,EXTRATO DE CONTRATO CONTRATO No 9020. ASSINATU...,"{'numero_contrato': ['9020'], 'data_assinatura..."
2,2,4_12.7.2019-R1,EXTRATO DO CONTRATO BRB 105/2019 Contratante: ...,"{'numero_contrato': ['105/2019'], 'partes_cont..."
3,3,1_31.5.2019-R2,EXTRATOS CONTRATUAIS Especie: Contrato no 018/...,"{'numero_contrato': ['018/2019-SES/DF'], 'part..."
4,4,1_31.5.2019-R4,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'processo_gdf': ['00070-00004138/2019-45'], '..."
...,...,...,...,...
754,754,638-R79,EXTRATO DO CONTRATO MÚLTIPLO DE PRESTAÇÃO DE S...,"{'partes_contratuais': ['SEEDF', 'EMPRESA BRAS..."
755,755,638-R80,EXTRATO DO CONTRATO DE AQUISIÇÃO DE BENS Nº 17...,"{'data_assinatura_contrato': ['15/03/2022'], '..."
756,756,638-R82,EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...,"{'partes_contratuais': ['SEEDF', 'SOLLAR ENGEN..."
757,757,638-R83,EXTRATO DO CONTRATO Nº 45.926/2022 Processo: 0...,"{'data_assinatura_contrato': ['15/03/2022'], '..."


In [ ]:
df_cp_f['texto'][0]

'EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No 45/2018 PROCESSO: 00070-00016600/2018-76.  Partes: SEAGRI/DF  e CIMAG  COMERCIO DE IMPLEMENTOS E MAQUINAS AGRICOLAS LTDA. Objeto: O Contrato tem por objeto a aquisicao dos seguintes itens: Item 20: Quantidade 03 ( tres). GRADE ARADORA CONTROLE R E M O TO , com 14 discos de 26 polegadas de diametro e 06 mm de espessura minima, mancais de rolamento com lubrificacao permanente em banho de oleo, ou a graxa, espacamento minimo entre os discos de 230mm. controle remoto para regulagem de profundidade do trabalho e transporte por meio do sistema hidraulico e pneus agricolas largura de corte minima de 1500 mm. compativel com tratores de potencia minima de 75 CV; Marca: CIMAG; Modelo / Versao: GRADE ARADORA CIMAG 14 x 26 x 6,0, consoante especifica o Edital de Pregao Eletronico no 04/2018 (Doc. Sei id 5507666) e a Proposta (Doc. Sei id14269745), que passam a integrar o presente Termo. Valor: O valor total do Contrato e de 39.600,00 (trinta e nove mi

In [ ]:
df_cp_f['label'][0]

"{'numero_contrato': ['45/2018'], 'processo_gdf': ['00070-00016600/2018-76'], 'partes_contratuais': ['SEAGRI/DF', 'CIMAG  COMERCIO DE IMPLEMENTOS E MAQUINAS AGRICOLAS LTDA'], 'valor_contrato': ['39.600,00'], 'nota_empenho': ['2018NE00463', '2018NE00464'], 'unidade_orcamentaria': ['14101'], 'programa_trabalho': ['20.606.6207.2889.0003'], 'natureza_despesa': ['449052'], 'data_assinatura_contrato': ['21/11/2018'], 'objeto_contrato': ['O Contrato tem por objeto a aquisicao dos seguintes itens: Item 20: Quantidade 03 ( tres). GRADE ARADORA CONTROLE R E M O TO , com 14 discos de 26 polegadas de diametro e 06 mm de espessura minima, mancais de rolamento com lubrificacao permanente em banho de oleo, ou a graxa, espacamento minimo entre os discos de 230mm. controle remoto para regulagem de profundidade do trabalho e transporte por meio do sistema hidraulico e pneus agricolas largura de corte minima de 1500 mm. compativel com tratores de potencia minima de 75 CV; Marca: CIMAG; Modelo / Versao: G

### Contando entidades

In [ ]:
Contagem_ents_preds = {'data_assinatura_contrato': 0,
                  'partes_contratuais': 0,
                  'natureza_despesa': 0,
                  'nota_empenho': 0,
                  'numero_contrato': 0,
                  'objeto_contrato': 0,
                  'processo_gdf': 0,
                  'programa_trabalho': 0,
                  'unidade_orcamentaria': 0,
                  'valor_contrato': 0,
                  'vigencia_contrato': 0
}

for index, row in df_cp_old.iterrows():
    dit = yaml.safe_load(row['pred'])
    for key,value in dit.items():
        Contagem_ents_preds[key]+=1

print(Contagem_ents_preds)

{'data_assinatura_contrato': 468, 'partes_contratuais': 778, 'natureza_despesa': 522, 'nota_empenho': 537, 'numero_contrato': 655, 'objeto_contrato': 771, 'processo_gdf': 583, 'programa_trabalho': 579, 'unidade_orcamentaria': 574, 'valor_contrato': 556, 'vigencia_contrato': 569}


In [ ]:
Contagem_ents_labels = {'data_assinatura_contrato':0,
                  'partes_contratuais':0,
                  'natureza_despesa':0,
                  'nota_empenho':0,
                  'numero_contrato':0,
                  'objeto_contrato':0,
                  'processo_gdf':0,
                  'programa_trabalho':0,
                  'unidade_orcamentaria':0,
                  'valor_contrato':0,
                  'vigencia_contrato':0
}

for index, row in df_cp_old.iterrows():
    dit = yaml.safe_load(row['label'])
    for key,value in dit.items():
        Contagem_ents_labels[key]+=1

print(Contagem_ents_labels)

{'data_assinatura_contrato': 501, 'partes_contratuais': 781, 'natureza_despesa': 469, 'nota_empenho': 488, 'numero_contrato': 777, 'objeto_contrato': 776, 'processo_gdf': 778, 'programa_trabalho': 508, 'unidade_orcamentaria': 411, 'valor_contrato': 706, 'vigencia_contrato': 750}


# Splitando dataset em treino-base, validação e teste

In [ ]:
training_data = df_cp_f.sample(frac=0.5, random_state=25)

df_base = df_cp_old
df_test = training_data
df_val = df_cp_f.drop(training_data.index)
# df_base = training_data.sample(n=800, random_state=25)
# df_train = training_data.drop(training_data.index)

df_base.reset_index(inplace=True,drop=True)
df_test.reset_index(inplace=True,drop=True)
df_val.reset_index(inplace=True,drop=True)

In [ ]:
df_base

,id,id_ato,texto,label,pred
0,0,12_22.11.2018-R2,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'partes_contratuais': ['SEAGRI/DF', 'CIMAG C...","{'natureza_despesa': ['449052'], 'partes_contr..."
1,1,12_22.11.2018-R3,EXTRATO DO CONTRATO No 32/2018 AQUISICAO DE BE...,"{'numero_contrato': ['32/2018'], 'processo_gdf...",{'partes_contratuais': ['INSTITUTO DE ASSIST. ...
2,2,12_22.11.2018-R7,"EXTRATO DO CONTRATO No 01/2018, NOS TERMOS DO ...","{'numero_contrato': ['01/2018'], 'processo_gdf...","{'natureza_despesa': ['449051'], 'partes_contr..."
3,3,12_22.11.2018-R5,EXTRATO DO CONTRATO PARA PRESTACAO DE SERVICOS...,"{'numero_contrato': ['07/2018'], 'processo_gdf...","{'natureza_despesa': ['3.3.90.39.11'], 'partes..."
4,4,12_22.11.2018-R6,EXTRATO DO CONTRATO TEC2B SOLUTIONS No 15/2018...,"{'partes_contratuais': ['IBRAM/DF', 'TEC2B SOL...","{'partes_contratuais': ['IBRAM/DF'], 'objeto_c..."
...,...,...,...,...,...
778,778,638-R75,EXTRATO DO CONTRATO BRB Nº 140/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Prosegur Brasi...","{'natureza_despesa': ['4'], 'partes_contratuai..."
779,779,638-R76,EXTRATO DO CONTRATO BRB Nº 141/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Renaforte Serv...",{'partes_contratuais': ['BRASILIA RESERVADO AO...
780,780,638-R78,EXTRATO DO CONTRATO BRB Nº 159/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Russell Bedfor...","{'natureza_despesa': ['4'], 'partes_contratuai..."
781,781,638-R81,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,"{'data_assinatura_contrato': ['15/03/2022'], '...","{'partes_contratuais': ['SEEDF', 'SEEDF'], 'ob..."


In [ ]:
df_val

,id,id_ato,texto,label
0,0,12_22.11.2018-R1,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'numero_contrato': ['45/2018'], 'processo_gdf..."
1,2,4_12.7.2019-R1,EXTRATO DO CONTRATO BRB 105/2019 Contratante: ...,"{'numero_contrato': ['105/2019'], 'partes_cont..."
2,8,1_31.5.2019-R8,EXTRATO DO CONTRATO DE PRESTACAO DE SERVICOS N...,"{'processo_gdf': ['220-00001674/2019-82', '24/..."
3,10,1_9.2.2018.1-R1,EXTRATO DE CONTRATO Processo: 001.000947/2017....,"{'processo_gdf': ['001.000947/2017'], 'numero_..."
4,11,1_9.2.2018.1-R4,EXTRATO DE INSTRUMENTO CONTRATUAL PROCESSO: 00...,"{'processo_gdf': ['00111-00010841/2017-70'], '..."
...,...,...,...,...
374,753,638-R77,EXTRATO DO CONTRATO BRB Nº 142/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Renaforte Serv..."
375,755,638-R80,EXTRATO DO CONTRATO DE AQUISIÇÃO DE BENS Nº 17...,"{'data_assinatura_contrato': ['15/03/2022'], '..."
376,756,638-R82,EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...,"{'partes_contratuais': ['SEEDF', 'SOLLAR ENGEN..."
377,757,638-R83,EXTRATO DO CONTRATO Nº 45.926/2022 Processo: 0...,"{'data_assinatura_contrato': ['15/03/2022'], '..."


In [ ]:
df_test

,id,id_ato,texto,label
0,444,631-R47,EXTRATO DO CONTRATO Nº 43338/2021 Processo: 04...,{'partes_contratuais': ['INSTITUTO DE ASSIST. ...
1,51,634-R93,EXTRATO DE CONTRATO Nº 45384/2021 Processo: 00...,"{'numero_contrato': ['45384/2021', '45384/2021..."
2,140,628-R103,EXTRATO DO CONTRATO Nº 204/2021 Processo: 0015...,"{'partes_contratuais': ['DF/SECEC/FAC', 'WAYNE..."
3,242,630-R76,EXTRATO DO CONTRATO DE PRESTAÇÃO DE SERVIÇO Nº...,"{'natureza_despesa': ['339039'], 'numero_contr..."
4,624,671-R49,EXTRATO DE CONTRATO DE PATROCÍNIO Extrato do C...,{'partes_contratuais': ['Companhia de Planejam...
...,...,...,...,...
375,273,637-R130,EXTRATO DO CONTRATO Nº 045005/2021 Processo: 0...,{'partes_contratuais': ['SECRETARIA DE ESTADO ...
376,594,626-R141,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS PEL...,"{'natureza_despesa': ['449052'], 'partes_contr..."
377,704,639-R143,EXTRATO DO CONTRATO Nº 43.980/2021 Processo: 0...,{'partes_contratuais': ['INSTITUTO DE ASSIST. ...
378,436,631-R35,EXTRATO DO CONTRATO Nº 42705/2021 Processo: 04...,{'partes_contratuais': ['INSTITUTO DE ASSIST. ...


In [ ]:
print('df_base:',len(df_base))
print('df_val:',len(df_val))
print('df_test:',len(df_test))

df_base: 783
df_val: 379
df_test: 380


### Contando para base, val e test

In [ ]:
Contagem_ents = {'data_assinatura_contrato': 0,
                  'partes_contratuais': 0,
                  'natureza_despesa': 0,
                  'nota_empenho': 0,
                  'numero_contrato': 0,
                  'objeto_contrato': 0,
                  'processo_gdf': 0,
                  'programa_trabalho': 0,
                  'unidade_orcamentaria': 0,
                  'valor_contrato': 0,
                  'vigencia_contrato': 0
}

for index, row in df_base.iterrows():
    dit = yaml.safe_load(row['pred'])
    for key,value in dit.items():
        Contagem_ents[key]+=1

print(Contagem_ents)

{'data_assinatura_contrato': 468, 'partes_contratuais': 778, 'natureza_despesa': 522, 'nota_empenho': 537, 'numero_contrato': 655, 'objeto_contrato': 771, 'processo_gdf': 583, 'programa_trabalho': 579, 'unidade_orcamentaria': 574, 'valor_contrato': 556, 'vigencia_contrato': 569}


In [ ]:
Contagem_ents = {'data_assinatura_contrato': 0,
                  'partes_contratuais': 0,
                  'natureza_despesa': 0,
                  'nota_empenho': 0,
                  'numero_contrato': 0,
                  'objeto_contrato': 0,
                  'processo_gdf': 0,
                  'programa_trabalho': 0,
                  'unidade_orcamentaria': 0,
                  'valor_contrato': 0,
                  'vigencia_contrato': 0
}

for index, row in df_base.iterrows():
    dit = yaml.safe_load(row['label'])
    for key,value in dit.items():
        Contagem_ents[key]+=1

print(Contagem_ents)

{'data_assinatura_contrato': 501, 'partes_contratuais': 781, 'natureza_despesa': 469, 'nota_empenho': 488, 'numero_contrato': 777, 'objeto_contrato': 776, 'processo_gdf': 778, 'programa_trabalho': 508, 'unidade_orcamentaria': 411, 'valor_contrato': 706, 'vigencia_contrato': 750}


In [ ]:
Contagem_ents = {'data_assinatura_contrato': 0,
                  'partes_contratuais': 0,
                  'natureza_despesa': 0,
                  'nota_empenho': 0,
                  'numero_contrato': 0,
                  'objeto_contrato': 0,
                  'processo_gdf': 0,
                  'programa_trabalho': 0,
                  'unidade_orcamentaria': 0,
                  'valor_contrato': 0,
                  'vigencia_contrato': 0
}

for index, row in df_val.iterrows():
    dit = yaml.safe_load(row['label'])
    for key,value in dit.items():
        Contagem_ents[key]+=1

print(Contagem_ents)

{'data_assinatura_contrato': 302, 'partes_contratuais': 379, 'natureza_despesa': 247, 'nota_empenho': 301, 'numero_contrato': 373, 'objeto_contrato': 379, 'processo_gdf': 378, 'programa_trabalho': 324, 'unidade_orcamentaria': 270, 'valor_contrato': 346, 'vigencia_contrato': 363}


In [ ]:
Contagem_ents = {'data_assinatura_contrato': 0,
                  'partes_contratuais': 0,
                  'natureza_despesa': 0,
                  'nota_empenho': 0,
                  'numero_contrato': 0,
                  'objeto_contrato': 0,
                  'processo_gdf': 0,
                  'programa_trabalho': 0,
                  'unidade_orcamentaria': 0,
                  'valor_contrato': 0,
                  'vigencia_contrato': 0
}

for index, row in df_test.iterrows():
    dit = yaml.safe_load(row['label'])
    for key,value in dit.items():
        Contagem_ents[key]+=1

print(Contagem_ents)

{'data_assinatura_contrato': 305, 'partes_contratuais': 380, 'natureza_despesa': 256, 'nota_empenho': 319, 'numero_contrato': 375, 'objeto_contrato': 377, 'processo_gdf': 377, 'programa_trabalho': 335, 'unidade_orcamentaria': 278, 'valor_contrato': 347, 'vigencia_contrato': 371}


# SALVALDO DATAFRAME

#### base GPT

In [ ]:
base_conll_GPT = ""
for index,row in df_base.iterrows():
  base_conll_GPT += tokenize(row['texto'],row['pred'],index)

2
3
5
6
10
25
26
27
33
37
39
40
41
42
43
48
60
61
64
66
72
73
76
77
78
80
82
83
84
99
102
106
107
109
110
114
115
117
118
120
124
125
142
148
149
151
152
155
156
159
160
162
163
164
166
167
170
172
178
179
180
181
182
187
190
194
201
202
204
218
219
220
221
225
226
228
229
231
232
233
238
239
242
245
248
249
262
263
264
277
282
290
293
294
297
299
301
302
303
305
308
310
315
317
318
319
320
321
322
328
337
350
351
352
353
357
369
370
384
385
387
388
389
390
397
403
404
405
406
410
411
412
429
433
434
440
442
445
446
447
450
451
462
463
468
477
479
486
489
491
492
495
496
497
498
499
500
505
508
510
512
515
518
519
521
522
529
531
534
535
537
538
541
542
543
544
547
548
549
551
552
553
554
557
567
574
579
582
583
584
585
586
589
590
600
601
602
606
608
612
613
620
630
632
635
642
644
649
651
655
656
657
658
659
660
661
703
704
705
706
708
709
711
714
715
719
721
722
726
731
748
749
752
753
756
758
766
768
772
773
778
779
780


In [ ]:
print(base_conll_GPT[0:20000])

EXTRATO O
DO O
CONTRATO O
PARA O
AQUISICAO O
DE O
BENS O
No O
47/2018 B-numero_contrato
PROCESSO O
: O
00070-00016822/ B-processo_gdf
2018-99 I-processo_gdf
. O
  O
Partes O
: O
SEAGRI B-partes_contratuais
/ I-partes_contratuais
DF I-partes_contratuais
  O
e O
CIMAG O
  O
COMERCIO O
DE O
IMPLEMENTOS O
E O
MAQUINAS O
AGRICOLAS O
LTDA O
. O
Objeto O
: O
O B-objeto_contrato
Contrato I-objeto_contrato
tem I-objeto_contrato
por I-objeto_contrato
objeto I-objeto_contrato
a I-objeto_contrato
aquisicao I-objeto_contrato
dos I-objeto_contrato
seguintes I-objeto_contrato
itens I-objeto_contrato
: I-objeto_contrato
Item I-objeto_contrato
20 I-objeto_contrato
: I-objeto_contrato
Quantidade I-objeto_contrato
05 I-objeto_contrato
( I-objeto_contrato
cinco I-objeto_contrato
) I-objeto_contrato
. I-objeto_contrato
GRADE I-objeto_contrato
ARADORA I-objeto_contrato
CONTROLE I-objeto_contrato
REMOTO I-objeto_contrato
, I-objeto_contrato
com I-objeto_contrato
14 I-objeto_contrato
discos I-objeto_contrato


#### base human

In [ ]:
base_conll_human = ""
for index,row in df_base.iterrows():
  base_conll_human += tokenize(row['texto'],row['label'],index)

69
71
75
86
114
115
116
118
119
124
126
141
142
143
144
151
152
154
155
156
159
161
163
164
165
166
167
168
175
176
178
185
193
194
197
200
201
202
204
218
219
220
221
222
228
229
230
232
233
235
238
239
245
249
254
259
297
298
305
306
307
308
309
311
312
315
316
317
318
319
320
321
403
404
405
406
410
427
428
429
430
431
440
444
445
446
463
464
472
479
547
548
549
550
552
557
559
561
565
567
583
584
585
586
587
589
590
600
601
602
606
609
612
613
731
758
761
766
767
768
769
770
771


In [ ]:
print(base_conll_human[0:20000])

EXTRATO O
DO O
CONTRATO O
PARA O
AQUISICAO O
DE O
BENS O
No O
47/2018 B-numero_contrato
PROCESSO O
: O
00070-00016822/ B-processo_gdf
2018-99 I-processo_gdf
. O
  O
Partes O
: O
SEAGRI B-partes_contratuais
/ I-partes_contratuais
DF I-partes_contratuais
  O
e O
CIMAG B-partes_contratuais
  I-partes_contratuais
COMERCIO I-partes_contratuais
DE I-partes_contratuais
IMPLEMENTOS I-partes_contratuais
E I-partes_contratuais
MAQUINAS I-partes_contratuais
AGRICOLAS I-partes_contratuais
LTDA I-partes_contratuais
. O
Objeto O
: O
O B-objeto_contrato
Contrato I-objeto_contrato
tem I-objeto_contrato
por I-objeto_contrato
objeto I-objeto_contrato
a I-objeto_contrato
aquisicao I-objeto_contrato
dos I-objeto_contrato
seguintes I-objeto_contrato
itens I-objeto_contrato
: I-objeto_contrato
Item I-objeto_contrato
20 I-objeto_contrato
: I-objeto_contrato
Quantidade I-objeto_contrato
05 I-objeto_contrato
( I-objeto_contrato
cinco I-objeto_contrato
) I-objeto_contrato
. I-objeto_contrato
GRADE I-objeto_cont

#### Val Conll

In [ ]:
val_conll = ""
for index,row in df_val.iterrows():
  val_conll += tokenize(row['texto'],row['label'],index)

25
32
60
64
67
76
94
95
96
97
104
116
117
118
119
121
125
127
149
157
163
174
201
203
206
264
265
273
274
275
284
287
350


In [ ]:
print(val_conll[0:20000])

EXTRATO O
DO O
CONTRATO O
PARA O
AQUISICAO O
DE O
BENS O
No O
45/2018 B-numero_contrato
PROCESSO O
: O
00070-00016600/ B-processo_gdf
2018-76 I-processo_gdf
. O
  O
Partes O
: O
SEAGRI B-partes_contratuais
/ I-partes_contratuais
DF I-partes_contratuais
  O
e O
CIMAG B-partes_contratuais
  I-partes_contratuais
COMERCIO I-partes_contratuais
DE I-partes_contratuais
IMPLEMENTOS I-partes_contratuais
E I-partes_contratuais
MAQUINAS I-partes_contratuais
AGRICOLAS I-partes_contratuais
LTDA I-partes_contratuais
. O
Objeto O
: O
O B-objeto_contrato
Contrato I-objeto_contrato
tem I-objeto_contrato
por I-objeto_contrato
objeto I-objeto_contrato
a I-objeto_contrato
aquisicao I-objeto_contrato
dos I-objeto_contrato
seguintes I-objeto_contrato
itens I-objeto_contrato
: I-objeto_contrato
Item I-objeto_contrato
20 I-objeto_contrato
: I-objeto_contrato
Quantidade I-objeto_contrato
03 I-objeto_contrato
( I-objeto_contrato
tres I-objeto_contrato
) I-objeto_contrato
. I-objeto_contrato
GRADE I-objeto_contr

#### Save all

In [ ]:
with open('/content/drive/MyDrive/tccF/base_conll_GPT.conll', 'w') as f:
    f.write(base_conll_GPT)

with open('/content/drive/MyDrive/tccF/base_conll_human.conll', 'w') as f:
    f.write(base_conll_human)

with open('/content/drive/MyDrive/tccF/val_conll.conll', 'w') as f:
    f.write(val_conll)

In [ ]:
df_base.to_csv('/content/drive/MyDrive/tccF/df_base.csv')
df_val.to_csv('/content/drive/MyDrive/tccF/df_val.csv')
df_test.to_csv('/content/drive/MyDrive/tccF/df_test.csv')

In [ ]:
# #conll_disc = ""
# for index,row in df_train.iterrows():
#   base_conll += tokenize(row['texto'],row['label'])
#   if index == 700:
#     break

In [ ]:
# base_conll

In [ ]:
# with open('/content/base_conll_human.conll', 'w') as f:
#     f.write(base_conll)

# with open('/content/drive/MyDrive/tcc/Datasets_tcc/model_tests/1500_base_conll_human.conll', 'w') as f:
#     f.write(base_conll)

# with open('/content/base_conll_human.conll', 'r') as f:
#     tes_len = f.read()

# tes_len=len(tes_len.split("\n\n"))
# print(tes_len)

In [ ]:
# with open('/content/base_conll_human.conll', 'r') as f:
#     tes_len = f.read()

# tes_len=len(tes_len.split("\n\n"))
# print(tes_len)

In [ ]:
with open('/content/drive/MyDrive/tccF/base_conll_GPT.conll', 'r') as f:
    tes_len = f.read()

tes_len=len(tes_len.split("\n\n"))
print(tes_len)

1064


In [ ]:
with open('/content/drive/MyDrive/tccF/base_conll_human.conll', 'r') as f:
    tes_len = f.read()

tes_len=len(tes_len.split("\n\n"))
print(tes_len)

907


In [ ]:
with open('/content/drive/MyDrive/tccF/val_conll.conll', 'r') as f:
    tes_len = f.read()

tes_len=len(tes_len.split("\n\n"))
print(tes_len)

413


# testando

In [ ]:
df_base = pd.read_csv('/content/drive/MyDrive/tccF/df_base.csv')
df_base.drop('Unnamed: 0', axis=1, inplace=True)
df_base

,id,id_ato,texto,label,pred
0,0,12_22.11.2018-R2,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'partes_contratuais': ['SEAGRI/DF', 'CIMAG C...","{'natureza_despesa': ['449052'], 'partes_contr..."
1,1,12_22.11.2018-R3,EXTRATO DO CONTRATO No 32/2018 AQUISICAO DE BE...,"{'numero_contrato': ['32/2018'], 'processo_gdf...",{'partes_contratuais': ['INSTITUTO DE ASSIST. ...
2,2,12_22.11.2018-R7,"EXTRATO DO CONTRATO No 01/2018, NOS TERMOS DO ...","{'numero_contrato': ['01/2018'], 'processo_gdf...","{'natureza_despesa': ['449051'], 'partes_contr..."
3,3,12_22.11.2018-R5,EXTRATO DO CONTRATO PARA PRESTACAO DE SERVICOS...,"{'numero_contrato': ['07/2018'], 'processo_gdf...","{'natureza_despesa': ['3.3.90.39.11'], 'partes..."
4,4,12_22.11.2018-R6,EXTRATO DO CONTRATO TEC2B SOLUTIONS No 15/2018...,"{'partes_contratuais': ['IBRAM/DF', 'TEC2B SOL...","{'partes_contratuais': ['IBRAM/DF'], 'objeto_c..."
...,...,...,...,...,...
778,778,638-R75,EXTRATO DO CONTRATO BRB Nº 140/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Prosegur Brasi...","{'natureza_despesa': ['4'], 'partes_contratuai..."
779,779,638-R76,EXTRATO DO CONTRATO BRB Nº 141/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Renaforte Serv...",{'partes_contratuais': ['BRASILIA RESERVADO AO...
780,780,638-R78,EXTRATO DO CONTRATO BRB Nº 159/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Russell Bedfor...","{'natureza_despesa': ['4'], 'partes_contratuai..."
781,781,638-R81,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,"{'data_assinatura_contrato': ['15/03/2022'], '...","{'partes_contratuais': ['SEEDF', 'SEEDF'], 'ob..."


In [ ]:
df_base['texto'][69]

'EXTRATO DE CONTRATO Nº 45400/2021 Processo: 00367-00000435/2021-47. Contrato: nº 45400/2021 - RA-FERC decorrente da Dispensa de Licitação (doc SEI 75903963). Firmado entre o Distrito Federal, por intermédio da Administração Regional da Fercal e a empresa FÊNIX INCORPORAÇÃO E SERVIÇOS LTDA, doravante denominada Contratada, CNPJ nº 27.548.454/0001-58, em 13/12/2021. Objeto: reforma dos banheiros na edificação da feira da Fercal/DF, conforme especificações técnicas apresentadas no processo 00367-00000435/2021-47. Valor total do contrato: R$ 96.843,20 (noventa e seis mil oitocentos e quarenta e três reais e vinte centavos). Unidade Gestora 190133, gestão 00001, unidade orçamentária 09135, programa de trabalho 15.451.6209.1110.9935, fonte de recurso 100000000; natureza da despesa 44.90.51. Vigência de 180 dias, contados de sua assinatura, com eficácia a partir da data da publicação do seu extrato no DODF. Legislação: Lei nº 8.666/93 e suas alterações. Partes: Pela Contratante: FERNANDO GUS

In [ ]:
df_base['pred'][69]

"{'partes_contratuais': ['DISTRITO FEDERAL', 'FÊNIX INCORPORAÇÃO E SERVIÇOS LTDA.'], 'objeto_contrato': ['reforma dos banheiros na edificação da feira da Fercal/DF, conforme especificações técnicas apresentadas no processo 00367-00000435/2021-47.'], 'unidade_orcamentaria': ['190133'], 'programa_trabalho': ['15.451.6209.1110.9935'], 'vigencia_contrato': ['180 dias'], 'numero_contrato': ['45400/2021'], 'processo_gdf': ['00367-00000435/2021-47'], 'valor_contrato': ['96.843,20'], 'nota_empenho': ['2021NE0/0244'], 'data_assinatura_contrato': ['13/12/2021']}"

In [ ]:
df_base['label'][69]

"{'data_assinatura_contrato': ['13/12/2021'], 'unidade_orcamentaria': ['09135'], 'programa_trabalho': ['15.451.6209.1110.9935'], 'natureza_despesa': ['44.90.51'], 'vigencia_contrato': ['Vigência de 180 dias, contados de sua assinatura, com eficácia a partir da data da publicação do seu extrato no DODF'], 'numero_contrato': ['45400/2021', '45400/2021'], 'processo_gdf': ['00367-00000435/2021-47', '00367-00000435/2021-47'], 'objeto_contrato': ['reforma dos banheiros na edificação da feira da Fercal/DF, conforme especificações técnicas apresentadas no processo 00367-00000435/2021-47'], 'partes_contratuais': ['RA-FERC', 'Administração Regional da Fercal', 'FÊNIX INCORPORAÇÃO E SERVIÇOS LTDA'], 'valor_contrato': ['96.843,20']}"

In [ ]:
preds = get_list_spans(df_base['texto'][69],yaml.safe_load(re.sub('\n', ' ', df_base['label'][69].replace(r'\n', '\n'))))

In [ ]:
preds

[[(355, 365, 'data_assinatura_contrato'),
  (687, 692, 'unidade_orcamentaria'),
  (715, 736, 'programa_trabalho'),
  (786, 794, 'natureza_despesa'),
  (796, 912, 'vigencia_contrato'),
  (23, 33, 'numero_contrato'),
  (81, 91, 'numero_contrato'),
  (44, 66, 'processo_gdf'),
  (375, 514, 'objeto_contrato'),
  (94, 101, 'partes_contratuais'),
  (210, 242, 'partes_contratuais'),
  (255, 289, 'partes_contratuais'),
  (544, 553, 'valor_contrato')],
 [(492, 514, 'processo_gdf'),
  (355, 365, 'data_assinatura_contrato'),
  (687, 692, 'unidade_orcamentaria'),
  (715, 736, 'programa_trabalho'),
  (786, 794, 'natureza_despesa'),
  (796, 912, 'vigencia_contrato'),
  (23, 33, 'numero_contrato'),
  (81, 91, 'numero_contrato'),
  (44, 66, 'processo_gdf'),
  (94, 101, 'partes_contratuais'),
  (210, 242, 'partes_contratuais'),
  (255, 289, 'partes_contratuais'),
  (544, 553, 'valor_contrato')]]

In [ ]:
df_base['texto'][86]

'EXTRATO DO CONTRATO Nº 28/2021 Processo: 00392-00010159/2021-54   Contratante: Companhia de Desenvolvimento Habitacional do Distrito Federal/CODHAB - CNPJ: 09.335.575/0001-30; Contratada: CAPITAL MIDIA SOLUÇÕES MOBILE   EIRELI - EPP, inscrita sob o CNPJ: 23.614.711/0001-51. Objeto: objeto desta contratação será a prestação de serviços de envio de torpedo SMS, sendo definidos segundo o interesse da Companhia de Desenvolvimento Habitacional do Distrito Federal - CODHAB na área de comunicação com o cidadão nos programas habitacionais com a necessidade específica de um segundo método para se comunicar e atingir o público alvo da Política Habitacional. Fundamentação Legal: Inciso II, do Art. 115, do Regulamento Interno de Licitações, Contratos e Convênios da CODHAB, em consonância com o Artigo 29, Inciso II, da Lei Federal nº 13.303/2016. Dotação Orçamentária: UO 28.209. Programa de Trabalho: 16.122.6208.4045.0005. Natureza da Despesa: 33.90.39. Fonte: 220. Nota de Empenho 2021NE00739, emi

In [ ]:
df_base['label'][86]

"{'natureza_despesa': ['33.90.39'], 'data_assinatura_contrato': ['05/11/2021'], 'numero_contrato': ['28/2021'], 'processo_gdf': ['00392-00010159/2021-54'], 'partes_contratuais': ['Companhia de Desenvolvimento Habitacional do Distrito Federal/CODHAB', 'Companhia de Desenvolvimento Habitacional do Distrito Federal - CODHAB', 'CAPITAL MIDIA SOLUÇÕES MOBILE   EIRELI - EPP'], 'objeto_contrato': ['objeto desta contratação será a prestação de serviços de envio de torpedo SMS, sendo definidos segundo o interesse da Companhia de Desenvolvimento Habitacional do Distrito Federal - CODHAB na área de comunicação com o cidadão nos programas habitacionais com a necessidade específica de um segundo método para se comunicar e atingir o público alvo da Política Habitacional'], 'unidade_orcamentaria': ['28.209'], 'programa_trabalho': ['16.122.6208.4045.0005'], 'nota_empenho': ['2021NE00739'], 'valor_contrato': ['50.000,00'], 'vigencia_contrato': ['12 (doze) meses']}"

In [ ]:
preds = get_list_spans(df_base['texto'][86],yaml.safe_load(re.sub('\n', ' ', df_base['label'][86].replace(r'\n', '\n'))))

In [ ]:
preds

[[(945, 953, 'natureza_despesa'),
  (1181, 1191, 'data_assinatura_contrato'),
  (23, 30, 'numero_contrato'),
  (41, 63, 'processo_gdf'),
  (79, 147, 'partes_contratuais'),
  (283, 654, 'objeto_contrato'),
  (871, 877, 'unidade_orcamentaria'),
  (901, 922, 'programa_trabalho'),
  (983, 994, 'nota_empenho'),
  (1030, 1041, 'valor_contrato'),
  (1203, 1218, 'vigencia_contrato')],
 [(401, 471, 'partes_contratuais'),
  (945, 953, 'natureza_despesa'),
  (1181, 1191, 'data_assinatura_contrato'),
  (23, 30, 'numero_contrato'),
  (41, 63, 'processo_gdf'),
  (79, 147, 'partes_contratuais'),
  (871, 877, 'unidade_orcamentaria'),
  (901, 922, 'programa_trabalho'),
  (983, 994, 'nota_empenho'),
  (1030, 1041, 'valor_contrato'),
  (1203, 1218, 'vigencia_contrato')]]